<a href="https://colab.research.google.com/github/nada-fathy/-pintOS/blob/master/Copy_of_DMass3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!pip install -q wordcloud
import wordcloud
import nltk
from sklearn import metrics
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn import metrics
nltk.download('punkt')
nltk.download('stopwords')
import re

TAG_RE = re.compile(r'<[^>]+>')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def preProcessing(filename):
  file = open(filename, 'rt')
  text = file.read()
  file.close()
  # lower cases
  text=text.lower()

  text ="".join([i for i in text if i not in string.punctuation])
  # split into sentences
 
  ###sentences = sent_tokenize(text)
  #print(sentences[0])

  # split into words

  tokens = word_tokenize(text)
  #print(tokens[:100])

  # remove all tokens that are not alphabetic
  
 # words = [word for word in tokens if word.isalpha()]
 # for word in words:
 #   word = TextBlob(word).correct()
    
  #print(words[:100])

 
  #####stop_words = stopwords.words('english')

  # filter out stop words
  
  stop_words = set(stopwords.words('english'))
  words = [w for w in tokens if not w in stop_words]
  #print(words[:100])

  # stemming of words
  
  porter = PorterStemmer()
  stemmed = [porter.stem(word) for word in words]
  #print(stemmed[:100])
  #print([' '.join(stemmed)])
  return ' '.join(stemmed)

In [5]:
import os
# assign directory
directory1 = '/content/drive/MyDrive/ca/aclImdb/train/neg'
directory2 = '/content/drive/MyDrive/ca/aclImdb/train/pos/'
Xtrain=[]
Ytrain=[]
# iterate over files in
# that directory
for filename in os.listdir(directory1):
    f = os.path.join(directory1, filename)
    # checking if it is a file
    if os.path.isfile(f):
      if len(Xtrain)%1000==0:
        print(len(Xtrain))
      #print(f)
      newText=preProcessing(f)
        #print(len(Xtrain))
        #print(newText)
      Xtrain.append(newText)
      name, extension = os.path.splitext(filename)
      Ytrain.append(int(name[-1]))
print(len(Xtrain))
print(len(Ytrain))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
12500
12500


In [6]:
for filename in os.listdir(directory2):
    f = os.path.join(directory2, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #print(f)
        if len(Xtrain)%1000==0:
          print(len(Xtrain))
        newText=preProcessing(f)
        #print(len(Xtrain))
        #print(newText)
        Xtrain.append(newText)
        name, extension = os.path.splitext(filename)
        Ytrain.append(int(name.split('_')[1]))
print(len(Xtrain))
print(len(Ytrain))

13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
25000


In [7]:
import os
# assign directory
directory1 = '/content/drive/MyDrive/ca/aclImdb/test/neg/'
directory2 = '/content/drive/MyDrive/ca/aclImdb/test/pos/'
Xtest=[]
Ytest=[]
# iterate over files in
# that directory
for filename in os.listdir(directory1):
    f = os.path.join(directory1, filename)
    # checking if it is a file
    if os.path.isfile(f):
      if len(Xtest)%1000==0:
        print(len(Xtest))
      text_file = open(f, "r")
      newText=text_file.read()
      Xtest.append(newText)
      name, extension = os.path.splitext(filename)
      Ytest.append(int(name[-1]))
print(len(Xtest))
print(len(Ytest))
for filename in os.listdir(directory2):
    f = os.path.join(directory2, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #print(f)
        if len(Xtest)%1000==0:
          print(len(Xtest))
        text_file = open(f, "r")
        newText=text_file.read()        
        #print(len(Xtrain))
        #print(newText)
        Xtest.append(newText)
        name, extension = os.path.splitext(filename)
        Ytest.append(int(name.split('_')[1]))
print(len(Xtest))
print(len(Ytest))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
12500
12500
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
25000


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
Xdf=pd.DataFrame(Xtrain)
# print(Xdf)
X=Xdf[0]
# print(xtrain)
# print(Xdf[0])
Ydf=pd.DataFrame(Ytrain)
Y=Ydf[0]
#x_train, x_dev, y_train, y_dev = train_test_split(X, Y, test_size=0.1, random_state=42)

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [9]:
# Create the param grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)


{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [ ]:
################################
def solveSVM(x_train, x_dev, y_train, y_dev):
  from sklearn.feature_extraction.text import TfidfVectorizer
  from sklearn.pipeline import Pipeline
  from sklearn.svm import LinearSVC
  from sklearn import metrics
  text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                      ('clf', LinearSVC()),])
  text_clf.fit(x_train, y_train)  
  predictions = text_clf.predict(x_dev)
  print(metrics.classification_report(y_dev,predictions)) 
  print(metrics.accuracy_score(y_dev,predictions))

In [ ]:
#################################
solveSVM(x_train, x_dev, y_train, y_dev)

              precision    recall  f1-score   support

           1       0.57      0.74      0.65       509
           2       0.30      0.17      0.22       239
           3       0.24      0.18      0.21       256
           4       0.35      0.30      0.32       294
           7       0.31      0.29      0.30       253
           8       0.31      0.30      0.30       285
           9       0.25      0.17      0.20       219
          10       0.48      0.64      0.55       445

    accuracy                           0.41      2500
   macro avg       0.35      0.35      0.34      2500
weighted avg       0.39      0.41      0.39      2500

0.414


In [9]:

def solveRandomforest(x_train, x_dev, y_train, y_dev):

  from sklearn.model_selection import RandomizedSearchCV
  rf_model=RandomForestClassifier()
  #rf_RandomGrid = RandomizedSearchCV(estimator = rf_Model, param_distributions = param_grid, cv = 10, verbose=2, n_jobs = 4)
  text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                       ('clf',RandomizedSearchCV(estimator = rf_model, param_distributions = param_grid, cv = 10, verbose=2, n_jobs = 4)),])
                     # ('clf', RandomForestClassifier(n_estimators=200, random_state=0)),])
  
  #rf_Grid = GridSearchCV(estimator =rf_model, param_grid = param_grid, cv = 10, verbose=2, n_jobs = 4)
  text_clf.fit(x_train, y_train)  
  
  #rf_Grid.best_params_
  #rf_RandomGrid.best_params_
  predictions = text_clf.predict(x_dev)
  print(metrics.classification_report(y_dev,predictions)) 
  print("Accuracy:",metrics.accuracy_score(y_dev,predictions))

In [11]:
solveRandomforest(Xtrain, Xtest, Ytrain, Ytest)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
              precision    recall  f1-score   support

           1       0.40      0.00      0.00      5022
           2       0.00      0.00      0.00      2302
           3       0.00      0.00      0.00      2541
           4       0.00      0.00      0.00      2635
           7       0.00      0.00      0.00      2307
           8       0.00      0.00      0.00      2850
           9       0.00      0.00      0.00      2344
          10       0.20      1.00      0.33      4999

    accuracy                           0.20     25000
   macro avg       0.08      0.13      0.04     25000
weighted avg       0.12      0.20      0.07     25000

Accuracy: 0.20004


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
################################

#from sklearn.feature_extraction.text import CountVectorizer
#count_vect = CountVectorizer()

#X_train_counts = count_vect.fit_transform(Xtrain)
#from sklearn.feature_extraction.text import TfidfTransformer
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(Xtrain) # remember to use the original X_train set
X_train_tfidf.shape

X_test_tfidf = vectorizer.fit_transform(Xtest) # remember to use the original X_train set


In [ ]:
#def solveRandomforest2(x_train, x_test, y_train, y_test):

 # from sklearn.model_selection import RandomizedSearchCV
  #rf_model=RandomForestClassifier()
  #rf_RandomGrid = RandomizedSearchCV(estimator = rf_model, param_distributions = param_grid, cv = 10, verbose=2, n_jobs = 4)
  #####text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     #####('clf',RandomizedSearchCV(estimator = rf_model, param_distributions = param_grid, cv = 10, verbose=2, n_jobs = 4)),])
                     ####('clf', RandomForestClassifier(n_estimators=200, random_state=0)),])
  
 ###### #rf_Grid = GridSearchCV(estimator =rf_model, param_grid = param_grid, cv = 10, verbose=2, n_jobs = 4)
  #rf_RandomGrid.fit(x_train, y_train)  
  
  #####rf_Grid.best_params_
  #rf_RandomGrid.best_params_
  #print (f'Train Accuracy - : {rf_RandomGrid.score(x_train,y_train):.3f}')
  #print (f'Test Accuracy - : {rf_RandomGrid.score(x_test,y_test):.3f}')
#solveRandomforest2(X_train_tfidf, Xtest, Ytrain, Ytest)

In [ ]:
###################################################

param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]



In [10]:
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]

def LogesticRegression(x_train,y_train,x_test,y_test):
  from sklearn.model_selection import RandomizedSearchCV
  rf_model=LogisticRegression()
  #rf_RandomGrid = RandomizedSearchCV(estimator = rf_Model, param_distributions = param_grid, cv = 10, verbose=2, n_jobs = 4)
  text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                       ('clf',RandomizedSearchCV(estimator = rf_model, param_distributions = param_grid, cv = 10, verbose=2, n_jobs = 1)),])
                     # ('clf', RandomForestClassifier(n_estimators=200, random_state=0)),])
  
  #rf_Grid = GridSearchCV(estimator =rf_model, param_grid = param_grid, cv = 10, verbose=2, n_jobs = 4)
  text_clf.fit(x_train, y_train)  
  
  #rf_Grid.best_params_
  #rf_RandomGrid.best_params_
  predictions = text_clf.predict(x_test)
  print(metrics.classification_report(y_test,predictions)) 
  print("Accuracy:",metrics.accuracy_score(y_test,predictions))

In [11]:

LogesticRegression(Xtrain, Ytrain, Xtest,  Ytest)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=newton-cg; total time=   0.0s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END C=545.5594781168514, max_iter=1000, penalty=l2, solver=lbfgs; total time= 4.3min


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END C=545.5594781168514, max_iter=1000, penalty=l2, solver=lbfgs; total time= 4.3min


...

[CV] END C=0.012742749857031334, max_iter=100, penalty=none, solver=newton-cg; total time= 1.2min


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1484: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio parameters"


[CV] END C=0.012742749857031334, max_iter=100, penalty=none, solver=newton-cg; total time= 1.2min


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solve

              precision    recall  f1-score   support

           1       0.62      0.13      0.22      5022
           2       0.13      0.35      0.19      2302
           3       0.16      0.01      0.02      2541
           4       0.22      0.00      0.01      2635
           7       0.13      0.47      0.20      2307
           8       0.16      0.20      0.17      2850
           9       0.13      0.00      0.01      2344
          10       0.33      0.37      0.35      4999

    accuracy                           0.20     25000
   macro avg       0.24      0.19      0.15     25000
weighted avg       0.28      0.20      0.17     25000

Accuracy: 0.20072


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.svm import SVC
# defining parameter range
param_grid = {'C': [0.1],
              'gamma': [1],
              'kernel': ['rbf']}
def solveSVM(x_train, x_dev, y_train, y_dev):
  
  text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                      ('clf', GridSearchCV(SVC(), param_grid, refit = True, cv = 10, verbose = 3)),])
  text_clf.fit(x_train, y_train)  
  
  #rf_Grid.best_params_
  #rf_RandomGrid.best_params_
  predictions = text_clf.predict(x_dev)
  print(metrics.classification_report(y_dev,predictions)) 
  print("Accuracy:",metrics.accuracy_score(y_dev,predictions))


In [11]:
solveSVM(Xtrain, Xtest, Ytrain,  Ytest)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.340 total time=17.1min
[CV 2/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.347 total time=18.5min
[CV 3/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.344 total time=17.7min
[CV 4/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.350 total time=18.6min
[CV 5/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.346 total time=17.0min
[CV 6/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.336 total time=16.7min
[CV 7/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.340 total time=16.9min
[CV 8/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.341 total time=17.1min
[CV 9/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.344 total time=17.3min
[CV 10/10] END ......C=0.1, gamma=1, kernel=rbf;, score=0.340 total time=18.5min
              precision    recall  f1-score   support

           1       0.68      0.05      0.10      5022
           2       0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
